In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import selenium
from selenium.webdriver.support.ui import WebDriverWait
import time
import pandas as pd
import numpy as np
import os
import re
import lxml

In [2]:
#load up the athlete page and accept the cookies tab
#driver.get("https://www.ufc.com/athletes/all")
#ele=driver.find_element_by_id("onetrust-accept-btn-handler")
#time.sleep(1)
#ele.click()
#time.sleep(2)

#to find the total of fighter
#total=BeautifulSoup(driver.page_source,"lxml")
#totalAth=total.find_all("div", {"class": "althelete-total"})
#stotal=re.search("\d+",str(totalAth)).group(0)
#stotal=int(int(stotal)/11)

#load up the fighter pages
#for i in range(stotal):
#    ele=driver.find_element_by_class_name("pager__item")
#    ele.click()
#    time.sleep(5)

#put the links into a data frame and out a csv
#total=BeautifulSoup(driver.page_source,"lxml")
#links=total.find_all("a", {"class": "e-button--black"})
#ufc = "https://www.ufc.com"
#links[0]["href"]
#linksAH=pd.DataFrame(np.nan, index=range(len(links)), columns=["link"],dtype="string")
#for i in range(len(links)):
#        linksAH.at[i,'link']= ufc +str(links[i]["href"])
#linksAH.to_csv("links_to_fighters.csv",index =False)


In [3]:
driver = webdriver.Chrome("C:\\Users\\bobby\\Desktop\\Python_project\\chromedriver.exe")
linksAH = pd.read_csv("links_to_fighters.csv")
columns = ["name","division","rank","str.ac","grap.ac","sig.str.land","sig.str.att","take.land","take.att",
           "sig.str.land.min","sig.str.abs.min","take.avg.15min","sub.avg.15min","sig.str.def",
          "take.def","knock.ratio","avg.time","sig.str.head","sig.str.head.percent","sig.str.body",
          "sig.str.body.percent","sig.str.leg","sig.str.leg.percent",
          "sig.str.position.stand","sig.str.position.clinch","sig.str.position.ground",
          "sig.str.position.stand.percent","sig.str.position.clinch.percent","sig.str.position.ground.percent",
           "win.ko","win.sub","win.dec","win.ko.percent","win.sub.percent","win.dec.percent",
          "wins","losses","age","leg.reach","reach","height","opponent","outcome"]
StatsOfFigthers=pd.DataFrame(columns =columns)
temp = pd.DataFrame(columns =columns,index =range(1))


In [4]:
for k in range(len(linksAH)):
    driver.get(linksAH.iloc[k,0])
    time.sleep(3)
    
    #accept cookie
    while True:
        try:
            ele=driver.find_element_by_id("onetrust-accept-btn-handler")
            ele.click()
            time.sleep(1)
            break
        except:
            break
        
    #clicks the "show stats button"
    try:
        time.sleep(5)
        ele = driver.find_element_by_class_name("stats-button__wrapper")
        ele.click()
        time.sleep(3)
    except:
        continue
    

    #loads all previous fights
    while True:
        try:
            ele = driver.find_element_by_class_name("pager__item")
            ele.click()
            time.sleep(5)
        except:
            break
    
    #get webpage
    time.sleep(3)
    stats = BeautifulSoup(driver.page_source,"lxml")
    
    
    record =str(stats.find(attrs={"class":"c-hero__headline-suffix"}))
    record=re.sub("<.*>","",record)
    record= re.sub(r"[^a-zA-Z0-9 -]","",record)
    record=re.sub(" ","",record)
    record=record.lower()
    
    win =re.search("[a-z]\d.*",record)
    win=re.sub("[a-z]","",str(win))
    win=re.search("\d+-",win).group(0)
    win=re.sub("-","",str(win))
    temp["wins"]=win
    
    loss =re.search("[a-z]\d.*",record)
    loss=re.sub("[a-z]","",str(loss))
    loss=re.search("-\d+",loss).group(0)
    loss = re.sub("-","",str(loss))
    temp["losses"]=loss
    
    division=re.search("[a-z]+",record).group(0)
    temp["division"]=division
    
    rank =re.search("\d+[a-z]",record).group(0)
    rank =re.sub("[a-z]","",rank)
    temp["rank"] = rank
    
    for a in stats.find_all(attrs ={"class":"c-overlap-athlete-detail__card"}):
        title =re.sub("<.*?>","",str(a.title))
        title =re.sub("\d+%","",title)
        title = title.strip()
        title = title.lower()
        if title == "striking accuracy":
            temp["str.ac"]=re.sub("<.*?>","",str(a.find_all(attrs={"class":"e-chart-circle__percent"})))
            temp["sig.str.land"] = re.sub("<.*?>","",str(a.find_all(attrs={"class":"c-overlap__stats-value"})[0]))
            temp["sig.str.att"]=re.sub("<.*?>","",str(a.find_all(attrs={"class":"c-overlap__stats-value"})[1]))
        if title == "grappling accuracy":
            temp["grap.ac"]=re.sub("<.*?>","",str(a.find_all(attrs={"class":"e-chart-circle__percent"})))
            temp["take.land"]=re.sub("<.*?>","",str(a.find_all(attrs={"class":"c-overlap__stats-value"})[0]))
            temp["take.att"]=re.sub("<.*?>","",str(a.find_all(attrs={"class":"c-overlap__stats-value"})[1]))
   
    
    for a in stats.find_all(attrs={"class":"c-stat-compare__group-1"}):
        label=re.sub("<.*?>","",str(a.find(attrs={"class":"c-stat-compare__label"})))
        num =re.sub("<.*?>","",str(a.find(attrs={"class":"c-stat-compare__number"})))
        label=label.strip()
        label=label.lower()
        if label == "sig. str. landed":
            temp["sig.str.land.min"]=num
        
        if label == "takedown avg":
            temp["take.avg.15min"]=num
        
        if label == "sig. str. defense":
            temp["sig.str.def"]= num
        
        if label == "knockdown ratio":
            temp["knock.ratio"]= num
        
    for a in stats.find_all(attrs={"class":"c-stat-compare__group-2"}):
        label=re.sub("<.*?>","",str(a.find(attrs={"class":"c-stat-compare__label"})))
        num =re.sub("<.*?>","",str(a.find(attrs={"class":"c-stat-compare__number"})))
        label=label.strip()
        label=label.lower()
        if label == "sig. str. absorbed":
            temp["sig.str.abs.min"]=num
        
        if label == "submission avg":
            temp["sub.avg.15min"]=num
        
        if label == "takedown defense":
            temp["take.def"]= num
        
        if label == "average fight time":
            temp["avg.time"]= num
    
    temp["sig.str.head"]=re.sub("<.*?>","",str(stats.find(attrs={"id":"e-stat-body_x5F__x5F_head_value"})))
    temp["sig.str.head.percent"]=re.sub("<.*?>","",str(stats.find(attrs={"id":"e-stat-body_x5F__x5F_head_percent"})))
    temp["sig.str.body"]=re.sub("<.*?>","",str(stats.find(attrs={"id":"e-stat-body_x5F__x5F_body_value"})))
    temp["sig.str.body.percent"]=re.sub("<.*?>","",str(stats.find(attrs={"id":"e-stat-body_x5F__x5F_body_percent"})))
    temp["sig.str.leg"]=re.sub("<.*?>","",str(stats.find(attrs={"id":"e-stat-body_x5F__x5F_leg_value"})))
    temp["sig.str.leg.percent"]=re.sub("<.*?>","",str(stats.find(attrs={"id":"e-stat-body_x5F__x5F_leg_percent"})))
   
    for a in stats.find_all(attrs={"class":"c-bio__field"}):
        lab=a.find(attrs={"class":"c-bio__label"})
        lab = re.sub("<.*?>","",str(lab))
        c=re.sub("<.*?>","",str(a))
        c= re.search("\d+",str(c))
        lab=lab.strip()
        lab = lab.lower()
        if(lab=="age"):
            temp["age"]=c.group(0)
        if(lab=="height"):
            temp["height"]=c.group(0)
        if(lab=="reach"):
            temp["reach"]=c.group(0)
        if(lab=="leg reach"):
            temp["leg.reach"]=c.group(0)
            
    for a in stats.find_all(attrs={"class":"c-stat-3bar__group"}):
        label =re.sub("<.*?>","",str(a.find(attrs={"class":"c-stat-3bar__label"})))
        num =re.sub("<.*?>","",str(a.find(attrs={"class":"c-stat-3bar__value"})))
        label=label.lower()
        label = label.strip()
        if label == "standing":
            temp["sig.str.position.stand"]=re.sub("\\(.+\\)","",num)
            temp["sig.str.position.stand.percent"]=re.findall("\\(.+\\)",num)
        if label == "clinch":
            temp["sig.str.position.clinch"]=re.sub("\\(.+\\)","",num)
            temp["sig.str.position.clinch.percent"]=re.findall("\\(.+\\)",num)
        if label == "ground":
            temp["sig.str.position.ground"]=re.sub("\\(.+\\)","",num)
            temp["sig.str.position.ground.percent"]=re.findall("\\(.+\\)",num)
        if label == "ko/tko":
            temp["win.ko"] = re.sub("\\(.+\\)","",num)
            temp["win.ko.percent"] = re.findall("\\(.+\\)",num)
        if label == "dec":
            temp["win.dec"] = re.sub("\\(.+\\)","",num)
            temp["win.dec.percent"] = re.findall("\\(.+\\)",num)
        if label == "sub":
            temp["win.sub"] = re.sub("\\(.+\\)","",num)
            temp["win.sub.percent"] = re.findall("\\(.+\\)",num)
    
    listofights =[]
    
    for a in stats.find_all(attrs={"class":"c-card-event--athlete-results__matchup"}):
        for j in a:
            listofights.append(j)
    
    for i in range(len(listofights)):
        for a in range(len(listofights)):
            try:
                if str(listofights[a]) == "\n":
                    listofights.pop(a)
            except:
                break
    
    listoffightnames =[]
    
    for a in stats.find_all(attrs={"class":"c-card-event--athlete-results__headline"}):
        listoffightnames.append(re.sub("<.*?>","",str(a)))
    try:
        name=re.search("-.*",str(linksAH.iloc[k,0])).group(0)
    except:
        continue
    
    name=re.sub("-","",name)
    name=name.strip()
    name=name.lower()
    temp["name"] = name
    try:
        temp=pd.concat([temp] *len(listoffightnames),ignore_index=True)
    except:
        continue
    
    for i in range(0,len(listofights),2):
    
        currentnames=listoffightnames[int(i/2)]
        currentnames=re.sub("vs","",currentnames)
        leftname=re.search(".*? +",currentnames).group(0)
        rightname=re.search(" +.*",currentnames).group(0)
        leftname=leftname.strip()
        leftname=leftname.lower()
        rightname=rightname.strip()
        rightname=rightname.lower()
    
        if leftname == name:
            temp.iloc[int(i/2)]["opponent"] = rightname
            if re.sub("<.*?>","",str(listofights[i])).strip() != "Win" and re.sub("<.*?>","",str(listofights[i+1])).strip() != "Win":
                temp.iloc[int(i/2)]["outcome"] = "upcoming"
            else:
                temp.iloc[int(i/2)]["outcome"]=re.sub("<.*?>","",str(listofights[i])).strip()
        else:
            temp.iloc[int(i/2)]["opponent"] = leftname
            if re.sub("<.*?>","",str(listofights[i])).strip() != "Win" and re.sub("<.*?>","",str(listofights[i+1])).strip() != "Win":
                temp.iloc[int(i/2)]["outcome"] = "upcoming"
            else:
                temp.iloc[int(i/2)]["outcome"]=re.sub("<.*?>","",str(listofights[i+1])).strip()
    
    StatsOfFigthers = StatsOfFigthers.append(temp,ignore_index = True)
    temp = temp.iloc[0:0]
    temp=temp.append(pd.Series(), ignore_index=True)
    

C:\Users\bobby\AppData\Local\Temp/ipykernel_14228/4287560299.py:218: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  temp=temp.append(pd.Series(), ignore_index=True)


In [5]:
StatsOfFigthers.to_csv("stats.csv",index=False)

55